In [37]:
import pandas as pd
from sqlalchemy import create_engine
from secrets import secrets

### Domains

In [2]:
df_domains = pd.read_csv('skills_db2/domain.csv').rename(columns={'Id': 'DomainId'})
df_domains.head()

,DomainId,Domain
0,0,Applied Science
1,1,Arts and Humanities
2,2,Business
3,3,Computer Science
4,4,Data Science


In [3]:
df_d2 = pd.DataFrame([
    [0, 'Business'],
    [1, 'Computer Science'],
    [2, 'Data Science'],
    [3, 'Information Technology (IT)'],
    [4, 'Others']
], columns=['Id', 'Domain'])
df_d2.head()

,Id,Domain
0,0,Business
1,1,Computer Science
2,2,Data Science
3,3,Information Technology (IT)
4,4,Others


### Skills

In [4]:
df_skills = pd.read_csv('skills_db2/skill.csv')
df_skills.head()

,Id,Skill,DomainId
0,0,Account Receivable,2
1,1,Accounting,2
2,2,Accounting Software,2
3,3,Accounting Standards,2
4,4,Accounts Payable,2


In [5]:
to_keep = ['Business', 'Computer Science', 'Data Science', 'Information Technology (IT)']

df_s2 = df_skills.merge(df_domains, left_on='DomainId', right_on='DomainId', how='left').drop(columns=['DomainId'])
df_s2.loc[~df_s2['Domain'].isin(to_keep), 'Domain'] = 'Others'
df_s2 = df_s2.merge(df_d2, on='Domain').drop(columns=['Domain']).rename(columns={'Id_x': 'Id', 'Id_y': 'DomainId'})
df_s2.head()

,Id,Skill,DomainId
0,0,Account Receivable,0
1,1,Accounting,0
2,2,Accounting Software,0
3,3,Accounting Standards,0
4,4,Accounts Payable,0


### New Skills

In [6]:
# New skills
new_skills = 'skills/Skills (from Indeed).xlsx'
df_new = pd.read_excel(new_skills, sheet_name='Skills')
df_new.head()

,Skill,Domain
0,Autodesk,Applied Science
1,SolidWorks,Applied Science
2,ANSYS,Applied Science
3,Research Methodology,Applied Science
4,Biotechnology Management,Applied Science


In [7]:
df_s3 = df_new.merge(df_d2, left_on='Domain', right_on='Domain', how='left').drop(columns=['Domain'])
df_s3 = df_s3.rename(columns={'Id': 'DomainId'})
df_s3 = df_s3.fillna(4)
df_s3['Id'] = df_s3.index + df_s2['Id'].max() + 1
df_s3.head()

,Skill,DomainId,Id
0,Autodesk,4.0,3241
1,SolidWorks,4.0,3242
2,ANSYS,4.0,3243
3,Research Methodology,4.0,3244
4,Biotechnology Management,4.0,3245


In [8]:
df_sfinal = df_s2.append(df_s3)
df_sfinal = df_sfinal.drop_duplicates()
df_sfinal.tail()

,Id,Skill,DomainId
38,3279,Network Address Translation (NAT),3.0
39,3280,Open Shortest Path First (OSPF),3.0
40,3281,ns-1,1.0
41,3282,ns-2,1.0
42,3283,ns-3,1.0


In [9]:
df_sfinal.astype({'Id': 'category', 'DomainId': 'category'}).describe()

,Id,Skill,DomainId
count,3284,3284,3284.0
unique,3284,3284,5.0
top,3283,Development Management,4.0
freq,1,1,988.0


### Redundant Skills

In [10]:
df_redskills = pd.read_excel('skills/Other Skills.xlsx')
df_redskills.head()

,Skill
0,Ada
1,Addition
2,Application
3,B
4,BASIC


In [11]:
df_r2 = df_redskills.merge(df_sfinal, on='Skill', how='left')
df_r2.loc[df_r2['Id'].isna()]

,Skill,Id,DomainId


In [12]:
df_r2 = df_r2[['Id']]
df_r2.head()

,Id
0,483
1,2270
2,522
3,534
4,537


### Alternate Skill Names

In [13]:
df_alternate = pd.read_excel('skills/Other Skills.xlsx', sheet_name='Duplicates')
df_alternate.head()

,Skill,Parent
0,NET,.NET Framework
1,AB Testing,A/B Testing
2,Accountancy,Accounting
3,Administrative,Administration
4,Macromedia Director,Adobe Director


In [14]:
df_a2 = df_alternate.merge(df_sfinal, left_on='Parent', right_on='Skill', how='left')
df_a2 = df_a2.rename(columns={'Skill_x': 'Skill'})
df_a2.loc[df_a2['Id'].isna()]

,Skill,Parent,Id,Skill_y,DomainId


In [15]:
df_a2 = df_a2[['Id', 'Skill']]
df_a2.head()

,Id,Skill
0,474,NET
1,478,AB Testing
2,1,Accountancy
3,7,Administrative
4,3275,Macromedia Director


### Source

In [22]:
df_source = pd.DataFrame([
    [0, 'Rubiqe'],
    [1, 'MYFutureJobs'],
    [2, 'Indeed']
], columns=['Id', 'Source'])
df_source.head()

,Id,Source
0,0,Rubiqe
1,1,MYFutureJobs
2,2,Indeed


### Skill Count

In [24]:
df_count = pd.read_csv('skills/ForAcceltic.csv')
df_count.head()

,Skill,Count,JobPostedMonth,Source
0,Sales,2580,2021-01-01,MYFutureJobs
1,Selection,190,2021-01-01,MYFutureJobs
2,Adobe Illustrator,63,2021-01-01,MYFutureJobs
3,Adobe Photoshop,94,2021-01-01,MYFutureJobs
4,Advertising,286,2021-01-01,MYFutureJobs


In [32]:
df_c2 = df_count.merge(df_sfinal[['Skill', 'Id']].rename(columns={'Id': 'skill_id'}), on=['Skill'], how='left')
df_c2 = df_c2.merge(df_source.rename(columns={'Id': 'source_id'}), on=['Source'], how='left')
df_c2.loc[df_c2['skill_id'].isna()]

,Skill,Count,JobPostedMonth,Source,skill_id,source_id


In [33]:
# df_c2['job_posted_date'] = pd.DatetimeIndex(df_c2['JobPostedMonth']).month
df_c2['JobPostedMonth'] = pd.to_datetime(df_c2['JobPostedMonth'])
df_c2['job_posted_date'] = df_c2['JobPostedMonth'].dt.strftime('%Y-%m-%d')
df_c2 = df_c2.drop(columns=['Skill', 'Source'])
df_c2 = df_c2.rename(columns={'Count': 'count', 'JobPostedMonth': 'job_posted_date_iso'})
df_c2 = df_c2.sort_values(by='skill_id')
df_c2 = df_c2.reset_index(drop=True)
df_c2['id'] = df_c2.index + 1
df_c2.head()

,count,job_posted_date_iso,skill_id,source_id,job_posted_date,id
0,122,2021-01-01,0,1,2021-01-01,1
1,769,2021-01-01,1,1,2021-01-01,2
2,91,2021-01-01,2,1,2021-01-01,3
3,51,2021-01-01,3,1,2021-01-01,4
4,132,2021-01-01,4,1,2021-01-01,5


In [34]:
df_c2.describe()

,count,skill_id,source_id,id
count,1035.000000,1035.000000,1035.0,1035.000000
mean,74.495652,1351.428986,1.0,518.000000
std,194.229208,989.401618,0.0,298.923067
min,1.000000,0.000000,1.0,1.000000
25%,8.000000,427.000000,1.0,259.500000
50%,17.000000,1114.000000,1.0,518.000000
75%,57.000000,2337.000000,1.0,776.500000
max,2580.000000,3251.000000,1.0,1035.000000


In [35]:
df_c2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   count                1035 non-null   int64         
 1   job_posted_date_iso  1035 non-null   datetime64[ns]
 2   skill_id             1035 non-null   int64         
 3   source_id            1035 non-null   int64         
 4   job_posted_date      1035 non-null   object        
 5   id                   1035 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 48.6+ KB


### To SQL

In [38]:
engine = create_engine(secrets['skills_db'])
df_d2.to_sql('Domain', engine, index=False, if_exists='replace')
df_sfinal.to_sql('Skill', engine, index=False, if_exists='replace')
df_source.to_sql('Source', engine, index=False, if_exists='replace')
df_r2.to_sql('IgnoreSkill', engine, index=False, if_exists='replace')
df_a2.to_sql('AlternateSkill', engine, index=False, if_exists='replace')

In [40]:
query = """
GRANT SELECT ON public."Domain" TO skills_reader;
GRANT SELECT ON public."Skill" TO skills_reader;
GRANT SELECT ON public."Source" TO skills_reader;
GRANT SELECT ON public."IgnoreSkill" TO skills_reader;
GRANT SELECT ON public."AlternateSkill" TO skills_reader;
"""
with engine.begin() as conn:
    conn.execute(query)

# Close connection
engine.dispose()

### Skill Count

In [41]:
engine = create_engine(secrets['skillstreet_dev'])
query = """
DELETE FROM skill_trend_details WHERE source_id = 1
"""
with engine.begin() as conn:
    conn.execute(query)
df_c2.to_sql('skill_trend_details', engine, index=False, if_exists='append')
engine.dispose()

In [42]:
engine = create_engine(secrets['skillstreet_stage'])
query = """
DELETE FROM skill_trend_details WHERE source_id = 1
"""
with engine.begin() as conn:
    conn.execute(query)
df_c2.to_sql('skill_trend_details', engine, index=False, if_exists='append')
engine.dispose()